In [1]:
import pandas as pd
import pandas_profiling
import numpy as np 
import json
import datetime
import re
import ast
from fastparquet import ParquetFile, write

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

### Data Folder Instructions

In [2]:
# Use this cell to specify the paths for the data folder in your local machines
# Use the variable 'datafolder' to specify the path
# Comment out all the data paths except your own
# Purple Air data ia assumed to be in a subfolder called 'purpleair' 
# NOAA data ia assumed to be in a subfolder called 'noaa' 
# For example, if the base data folder is '/users/data', purpleair data should be in '/users/data/purpleair'

# Angshuman's local path
datafolder = "/Users/apaul2/Documents/_Common/capstone/Project/data"

### Combine PurpleAir and NOAA data

In [3]:
def createHashKey(row):
    if np.isnan(row['lat']):
        str_lat = ''
    else:
        str_lat = str(row['lat'])
        
        
    if np.isnan(row['lon']):
        str_lon = ''
    else:
        str_lon = str(row['lon'])
        
    return hash(str_lat + str_lon)

In [4]:
# Read from noaa data that was stored earlier
sep2019_noaa_df = pd.read_parquet("{}/noaa/bay_Sep2019_withloc.parquet".format(datafolder))
sep27_noaa_df = sep2019_noaa_df[sep2019_noaa_df.date == '09/27/19']

In [5]:
sep27_noaa_df['datetime'] = sep27_noaa_df[['year', 'month','day','hour','minute']].apply(lambda x: int(''.join(x)), axis=1)

In [6]:
sep27_noaa_df.drop(['year', 'month','day','hour','minute','date','timestamp'], axis=1, inplace=True)

In [7]:
sep27_noaa_df.head()

,wban_number,call_sign,call_sign2,rec_length,interval,call_sign3,zulu_time,report_modifier,wind_data,wind_direction,wind_speed,gusts,gust_speed,variable_winds,variable_wind_info,sys_maint_reqd,num_fields,lat,lon,datetime
6745,23234,KSFO,SFO,103,5-MIN,KSFO,270800Z,,True,250,6.0,False,NaN,False,,False,18,37.62,-122.365,201909270000
6746,23234,KSFO,SFO,103,5-MIN,KSFO,270805Z,,True,250,7.0,False,NaN,False,,False,18,37.62,-122.365,201909270005
6747,23234,KSFO,SFO,103,5-MIN,KSFO,270810Z,,True,240,8.0,False,NaN,False,,False,18,37.62,-122.365,201909270010
6748,23234,KSFO,SFO,102,5-MIN,KSFO,270815Z,,True,250,5.0,False,NaN,False,,False,18,37.62,-122.365,201909270015
6749,23234,KSFO,SFO,103,5-MIN,KSFO,270820Z,,True,250,6.0,False,NaN,False,,False,18,37.62,-122.365,201909270020


In [8]:
sep27_noaa_df.wban_number.count(), sep27_noaa_df.lat.nunique(), sep27_noaa_df.lon.nunique()

(1728, 6, 6)

In [10]:
# Read epa data from file
sep27_epa_df = pd.read_parquet("{}/ambient/epa_sep27_2019.parquet".format(datafolder))
sep27_epa_df['createdhr'] = sep27_epa_df['created'].apply(lambda x: int(str(x)[:-2]))  # date key at hour level as the data is hourly
sep27_epa_df.head()

,lat,lon,utc,parameter,epa_pm25_unit,epa_pm25_value,raw_concentration,aqi,category,site_name,agency_name,full_aqs_code,intl_aqs_code,created,createdhr
index,,,,,,,,,,,,,,,
55073,37.972200,-122.518900,2019-09-27 07:00:00,PM2.5,UG/M3,8.2,7.0,34,1,San Rafael,San Francisco Bay Area AQMD,60410001,840060410001,201909270000,2019092700
55074,37.765800,-122.397800,2019-09-27 07:00:00,PM2.5,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005,201909270000,2019092700
55075,37.960400,-122.357100,2019-09-27 07:00:00,PM2.5,UG/M3,8.4,8.0,35,1,San Pablo - Rumrill,San Francisco Bay Area AQMD,60131004,840060131004,201909270000,2019092700
55076,37.864767,-122.302741,2019-09-27 07:00:00,PM2.5,UG/M3,7.0,6.0,29,1,Berkeley Aquatic Park,San Francisco Bay Area AQMD,60010013,840060010013,201909270000,2019092700
55077,37.814800,-122.282402,2019-09-27 07:00:00,PM2.5,UG/M3,9.2,8.0,38,1,Oakland West,San Francisco Bay Area AQMD,60010011,840060010011,201909270000,2019092700


In [11]:
# Read purple air data from file
bay_ts_df = pd.read_parquet("{}/purpleair/ts_0927_withaddress_final.parquet".format(datafolder))
bay_ts_df['createdhr'] = bay_ts_df['created'].apply(lambda x: int(str(x)[:-2]))  # date key at hour level to join with hourly epa data
bay_ts_df.head()

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,created,pm1_0_atm,pm2_5_atm,pm10_0_atm,uptime,rssi,temperature,humidity,pm2_5_cf_1,a_h,device_loc_typ,high_reading_flag,hidden,sensor_id,sensor_name,lat,lon,parent_id,is_owner,city,county,zipcode,created_at,year,month,day,hour,minute,createdhr
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,935.56,270.69,45.63,7.76,2.35,0.00,4.57,8.82,201909270000,4.34,7.07,9.19,913.0,-74.0,83.0,46.0,7.07,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:00,2019,09,27,00,00,2019092700
1,842.38,244.69,51.99,9.53,3.35,0.00,3.78,8.69,201909270010,4.24,6.82,9.66,923.0,-71.0,82.0,47.0,6.82,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:10,2019,09,27,00,10,2019092700
2,832.65,247.20,41.00,6.56,2.14,1.10,3.93,8.15,201909270020,4.07,7.60,9.65,932.0,-72.0,81.0,47.0,7.60,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:20,2019,09,27,00,20,2019092700
3,916.69,260.89,48.72,7.73,2.65,0.00,3.97,8.58,201909270030,3.97,6.94,8.58,942.0,-75.0,81.0,48.0,6.94,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:30,2019,09,27,00,30,2019092700
4,924.00,269.25,51.26,6.03,3.65,1.59,4.68,9.53,201909270040,4.68,6.85,9.53,952.0,-73.0,80.0,49.0,6.85,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:40,2019,09,27,00,40,2019092700


In [12]:
bay_ts_df.sensor_id.count(), bay_ts_df.sensor_id.nunique(), bay_ts_df.lat.nunique(), bay_ts_df.lon.nunique()

(73660, 520, 298, 298)

In [13]:
# Add lat-lon based hashes to noaa and purple air dataframes
# bay_stations_df['asoslatlonhash'] = bay_stations_df.apply (lambda row: createHashKey(row), axis=1)
bay_ts_df['tslatlonhash'] = bay_ts_df.apply (lambda row: createHashKey(row), axis=1)
sep27_noaa_df['asoslatlonhash'] = sep27_noaa_df.apply (lambda row: createHashKey(row), axis=1)
sep27_epa_df['epalatlonhash'] = sep27_epa_df.apply (lambda row: createHashKey(row), axis=1)

In [14]:
# Keep only the asos columns needed to determine the lat-lon mapping
Sep27_noaa_latlon_df = sep27_noaa_df[['asoslatlonhash','lat','lon']]
Sep27_noaa_latlon_df.drop_duplicates(inplace=True)
Sep27_noaa_latlon_df.asoslatlonhash.count(), Sep27_noaa_latlon_df.asoslatlonhash.nunique()

(6, 6)

In [15]:
# Keep only the epa columns needed to determine the lat-lon mapping
Sep27_epa_latlon_df = sep27_epa_df[['epalatlonhash','lat','lon']]
Sep27_epa_latlon_df.drop_duplicates(inplace=True)
Sep27_epa_latlon_df.epalatlonhash.count(), Sep27_epa_latlon_df.epalatlonhash.nunique()

(6, 6)

In [16]:
# Keep only the purple air columns needed to determine the lat-lon mapping
usa_purple_latlon_df = bay_ts_df[['tslatlonhash','lat','lon']]
usa_purple_latlon_df.drop_duplicates(inplace=True)
usa_purple_latlon_df.tslatlonhash.count(), usa_purple_latlon_df.tslatlonhash.nunique()

(298, 298)

In [17]:
Sep27_noaa_latlon_df.set_index('asoslatlonhash', inplace=True)
usa_purple_latlon_df.set_index('tslatlonhash', inplace=True)
Sep27_epa_latlon_df.set_index('epalatlonhash', inplace=True)

In [18]:
# Find the closest asos lat-lon mapping corresponding to the purple air records
closest_asos_points = {}
for name, point in usa_purple_latlon_df.iterrows():
#     print(name, point)
#     break
    distances = (((Sep27_noaa_latlon_df - point) ** 2).sum(axis=1)**.5)
    closest_asos_points[name] = distances.sort_values().index[0]
    
# Create dataframe from lat-lon mapping
asoslatlonmap_df = pd.DataFrame(list(closest_asos_points.items()), columns=['tslatlonhash','asoslatlonhash'])
asoslatlonmap_df.count()

tslatlonhash      298
asoslatlonhash    298
dtype: int64

In [19]:
# Find the closest asos lat-lon mapping corresponding to the purple air records
closest_epa_points = {}
for name, point in usa_purple_latlon_df.iterrows():
#     print(name, point)
#     break
    distances = (((Sep27_epa_latlon_df - point) ** 2).sum(axis=1)**.5)
    closest_epa_points[name] = distances.sort_values().index[0]
    
# Create dataframe from lat-lon mapping
epalatlonmap_df = pd.DataFrame(list(closest_epa_points.items()), columns=['tslatlonhash','epalatlonhash'])
epalatlonmap_df.count()

tslatlonhash     298
epalatlonhash    298
dtype: int64

In [20]:
# Merge purple air data to lat-lon mappings first and then 
# merge the resulting dataframe to asos and epa dataframes
merged_df = pd.merge(bay_ts_df, asoslatlonmap_df, on='tslatlonhash')
merged_df = pd.merge(merged_df, epalatlonmap_df, on='tslatlonhash')

In [21]:
merged_df.sensor_id.count()

73660

In [22]:
merged_df.head()

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,created,pm1_0_atm,pm2_5_atm,pm10_0_atm,uptime,rssi,temperature,humidity,pm2_5_cf_1,a_h,device_loc_typ,high_reading_flag,hidden,sensor_id,sensor_name,lat,lon,parent_id,is_owner,city,county,zipcode,created_at,year,month,day,hour,minute,createdhr,tslatlonhash,asoslatlonhash,epalatlonhash
0,935.56,270.69,45.63,7.76,2.35,0.00,4.57,8.82,201909270000,4.34,7.07,9.19,913.0,-74.0,83.0,46.0,7.07,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:00,2019,09,27,00,00,2019092700,2549551147210232084,-2433840552098672732,4608761411959698118
1,842.38,244.69,51.99,9.53,3.35,0.00,3.78,8.69,201909270010,4.24,6.82,9.66,923.0,-71.0,82.0,47.0,6.82,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:10,2019,09,27,00,10,2019092700,2549551147210232084,-2433840552098672732,4608761411959698118
2,832.65,247.20,41.00,6.56,2.14,1.10,3.93,8.15,201909270020,4.07,7.60,9.65,932.0,-72.0,81.0,47.0,7.60,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:20,2019,09,27,00,20,2019092700,2549551147210232084,-2433840552098672732,4608761411959698118
3,916.69,260.89,48.72,7.73,2.65,0.00,3.97,8.58,201909270030,3.97,6.94,8.58,942.0,-75.0,81.0,48.0,6.94,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:30,2019,09,27,00,30,2019092700,2549551147210232084,-2433840552098672732,4608761411959698118
4,924.00,269.25,51.26,6.03,3.65,1.59,4.68,9.53,201909270040,4.68,6.85,9.53,952.0,-73.0,80.0,49.0,6.85,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:40,2019,09,27,00,40,2019092700,2549551147210232084,-2433840552098672732,4608761411959698118


In [23]:
# Drop common and unwanted columns from noaa and epa dataframes
sep27_noaa_df.drop(['lat','lon'], axis=1, inplace=True)
sep27_epa_df.drop(['lat','lon'], axis=1, inplace=True)

In [26]:
# Combine asos data
combined_df = pd.merge(merged_df, sep27_noaa_df,  how='left', left_on=['asoslatlonhash', 'created'], right_on=['asoslatlonhash', 'datetime'])

# Combine epa data
combined_df = pd.merge(combined_df, sep27_epa_df,  how='left', left_on=['epalatlonhash', 'createdhr'], right_on=['epalatlonhash', 'createdhr'])

# # Drop unwanted columns
combined_df.drop(['tslatlonhash', 'asoslatlonhash', 'epalatlonhash', 'rec_length','num_fields', 'datetime', 'utc', 'parameter', 'createdhr','created_y'], axis=1, inplace=True)

In [8]:
combined_df.columns = ['0_3um', '0_5um', '1_0um', '2_5um', '5_0um', '10_0um', 'pm1_0','pm10_0', 'created', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm', 'uptime','rssi', 
                       'temperature', 'humidity', 'pm2_5_cf_1', 'a_h', 'device_loc_typ', 'high_reading_flag', 'hidden', 'sensor_id', 'sensor_name', 'lat', 'lon', 'parent_id', 
                       'is_owner', 'city', 'county', 'zipcode', 'created_at', 'year', 'month', 'day', 'hour', 'minute', 'wban_number', 'call_sign', 'call_sign2', 'interval', 
                       'call_sign3', 'zulu_time', 'report_modifier', 'wind_data', 'wind_direction', 'wind_speed', 'gusts', 'gust_speed', 'variable_winds', 'variable_wind_info', 
                       'sys_maint_reqd', 'epa_pm25_unit', 'epa_pm25_value', 'raw_concentration', 'aqi', 'category', 'site_name', 'agency_name', 'full_aqs_code', 'intl_aqs_code']

In [9]:
combined_df.head()

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,created,pm1_0_atm,pm2_5_atm,pm10_0_atm,uptime,rssi,temperature,humidity,pm2_5_cf_1,a_h,device_loc_typ,high_reading_flag,hidden,sensor_id,sensor_name,lat,lon,parent_id,is_owner,city,county,zipcode,created_at,year,month,day,hour,minute,wban_number,call_sign,call_sign2,interval,call_sign3,zulu_time,report_modifier,wind_data,wind_direction,wind_speed,gusts,gust_speed,variable_winds,variable_wind_info,sys_maint_reqd,epa_pm25_unit,epa_pm25_value,raw_concentration,aqi,category,site_name,agency_name,full_aqs_code,intl_aqs_code
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,935.56,270.69,45.63,7.76,2.35,0.00,4.57,8.82,201909270000,4.34,7.07,9.19,913.0,-74.0,83.0,46.0,7.07,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:00,2019,09,27,00,00,23234,KSFO,SFO,5-MIN,KSFO,270800Z,,1.0,250,6.0,0.0,NaN,0.0,,0.0,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
1,842.38,244.69,51.99,9.53,3.35,0.00,3.78,8.69,201909270010,4.24,6.82,9.66,923.0,-71.0,82.0,47.0,6.82,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:10,2019,09,27,00,10,23234,KSFO,SFO,5-MIN,KSFO,270810Z,,1.0,240,8.0,0.0,NaN,0.0,,0.0,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
2,832.65,247.20,41.00,6.56,2.14,1.10,3.93,8.15,201909270020,4.07,7.60,9.65,932.0,-72.0,81.0,47.0,7.60,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:20,2019,09,27,00,20,23234,KSFO,SFO,5-MIN,KSFO,270820Z,,1.0,250,6.0,0.0,NaN,0.0,,0.0,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
3,916.69,260.89,48.72,7.73,2.65,0.00,3.97,8.58,201909270030,3.97,6.94,8.58,942.0,-75.0,81.0,48.0,6.94,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:30,2019,09,27,00,30,23234,KSFO,SFO,5-MIN,KSFO,270830Z,,1.0,250,7.0,0.0,NaN,0.0,,0.0,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005
4,924.00,269.25,51.26,6.03,3.65,1.59,4.68,9.53,201909270040,4.68,6.85,9.53,952.0,-73.0,80.0,49.0,6.85,None,outside,nan,false,16939,#SAFQ11,37.72244,-122.439302,nan,0,San Francisco,San Francisco County,94112,2019/09/27T00:40,2019,09,27,00,40,23234,KSFO,SFO,5-MIN,KSFO,270840Z,,1.0,250,7.0,0.0,NaN,0.0,,0.0,UG/M3,5.8,4.0,24,1,San Francisco,San Francisco Bay Area AQMD,60750005,840060750005


In [10]:
# Write to file
parquet_file = "{}/20190927.parquet".format(datafolder)
write(parquet_file, combined_df,compression='GZIP')